# Análise de resultados do IMI - Feedback da sessão de julgamento

Este notebook procura fazer a análise das respostas da pesquisa de feedback da sessão de julgamento do TST. Para isso, as seguintes ações serão realizadas:

- Recuperação das perguntas formuladas, para mapeamento das questões conforme o id do IMI
- Identificação de quais perguntas são reversas (Tp = R)
- Mapeamento das respostas para cada pergunta
- Cálculo do valor da resposta das perguntas reversas seguindo a fórmula: (Valor máximo Escala de Likert + 1 ) - Valor da resposta
- Cálculo da confiabilidade com o Alpha de Cronbach
- Cálculo da Média e Desvio Padrão
- Similaridade: Correlação de Pearson

## Importação das bibliotecas

Utilizaremos as seguintes bibliotecas:
- pandas: Permite criar DataFrames a partir dos arquivos Excel
- numpy: Biblioteca estatítica do Python
- pingouin: Lib que permite o cálculo do Alpha de Cronbach

In [112]:
import pandas as pd
import numpy as np
import pingouin as pg

## Recuperação das perguntas

Abriremos o arquivo `imi-perguntas.xlsx` para criar os DataFrames:

In [113]:
perguntasImi = pd.read_excel('arquivos/imi-perguntas.xlsx', sheet_name='IMI Grupo 1')
print(perguntasImi)

    Id Sc   Tp                               Pergunta (Português)  \
0    1  I  NaN      Tive muito prazer em realizar esta atividade.   
1    2  I  NaN             Foi divertido realizar esta atividade.   
2    3  I    R                     Achei esta atividade monótona.   
3    4  I    R      Não prestei qualquer atenção nesta atividade.   
4    5  I  NaN  Eu descreveria esta atividade como muito inter...   
5    8  P  NaN      Sou razoavelmente competente nesta atividade.   
6    9  P  NaN  Creio ter me saído muito bem nesta atividade, ...   
7   11  P  NaN  Estou satisfeito com o meu desempenho nesta at...   
8   13  P    R  Esta foi uma atividade que eu não poderia faze...   
9   14  E  NaN       Fiz um esforço considerável nesta atividade.   
10  15  E    R  Não me esforcei muito para ter um bom desempen...   
11  16  E  NaN           Eu me esforcei bastante nesta atividade.   
12  17  E  NaN  Foi importante para mim ter um bom desempenho ...   
13  19  T    R  Não fiquei nem um 

Recuperaremos apenas as colunas de interesse: ID, Sc, Tp, Perguntas adaptadas com ajustes para envio

In [114]:
dfPerguntas = pd.DataFrame().assign(id=perguntasImi['Id'], sc=perguntasImi['Sc'], tp=perguntasImi['Tp'],perguntaAdaptada=perguntasImi['Perguntas adaptadas com ajustes para envio'], perguntaOriginal=perguntasImi['Pergunta (Português)'])
print(dfPerguntas)

    id sc   tp                                   perguntaAdaptada  \
0    1  I  NaN  Foi prazeroso ajudar o TST com a pesquisa de f...   
1    2  I  NaN                                                NaN   
2    3  I    R  Achei monótono responder a pesquisa de feedbac...   
3    4  I    R                                                NaN   
4    5  I  NaN  Eu achei que responder à pesquisa de feedback ...   
5    8  P  NaN  Sou razoavelmente competente para sugerir melh...   
6    9  P  NaN                                                NaN   
7   11  P  NaN                                                NaN   
8   13  P    R  Sugerir melhorias com pesquisas de feedback pa...   
9   14  E  NaN  Fiz um esforço considerável em responder a pes...   
10  15  E    R                                                NaN   
11  16  E  NaN                                                NaN   
12  17  E  NaN                                                NaN   
13  19  T    R                    

Agora, filtraremos o dataset para trazer apenas os registros que possuem perguntas adaptadas:

In [115]:
dfPerguntasFiltradas = dfPerguntas.query('perguntaAdaptada.notnull()')
print(dfPerguntasFiltradas)

    id sc   tp                                   perguntaAdaptada  \
0    1  I  NaN  Foi prazeroso ajudar o TST com a pesquisa de f...   
2    3  I    R  Achei monótono responder a pesquisa de feedbac...   
4    5  I  NaN  Eu achei que responder à pesquisa de feedback ...   
5    8  P  NaN  Sou razoavelmente competente para sugerir melh...   
8   13  P    R  Sugerir melhorias com pesquisas de feedback pa...   
9   14  E  NaN  Fiz um esforço considerável em responder a pes...   
15  21  T  NaN  Fiquei bem à vontade enquanto respondia a pesq...   
17  23  T  NaN  Eu me senti pressionado(a) enquanto respondia ...   
23  29  C  NaN  Respondi a pesquisa de feedback para o TST por...   
24  30  C    R  Respondi a pesquisa de feedback para o TST por...   
25  32  V  NaN  Acho que responder a pesquisa de feedback é út...   
27  35  V  NaN  Creio que responder a pesquisa de feedback par...   
31  40  R  NaN  Senti que posso confiar nas pesquisa de feedba...   
32  41  R  NaN  Gostaria de intera

## Recuperação das respostas

Faremos agora o carregamento das respostas enviadas do formulário:

In [116]:
dfRespostas = pd.read_excel('arquivos/respostas-questionario.xlsx')
print(dfRespostas)

      Carimbo de data/hora  \
0  2022-12-08 11:09:37.891   
1  2022-12-08 11:17:20.247   
2  2022-12-08 11:24:41.264   
3  2022-12-08 11:38:58.288   
4  2022-12-08 11:49:07.551   
5  2022-12-08 13:20:46.608   
6  2022-12-08 14:11:07.361   
7  2022-12-08 15:08:20.335   
8  2022-12-08 15:12:21.118   
9  2022-12-08 19:23:04.912   
10 2022-12-09 08:45:03.231   
11 2022-12-09 10:33:57.132   
12 2022-12-09 11:18:33.257   
13 2022-12-11 06:38:50.143   
14 2022-12-12 06:20:09.375   

    Eu achei que responder à pesquisa de feedback de sessão de julgamento foi muito interessante.  \
0                                                   4                                               
1                                                   5                                               
2                                                   3                                               
3                                                   5                                               
4             

In [117]:
dfRespostas = dfRespostas.drop(columns=['Carimbo de data/hora'])
print(dfRespostas)

    Eu achei que responder à pesquisa de feedback de sessão de julgamento foi muito interessante.  \
0                                                   4                                               
1                                                   5                                               
2                                                   3                                               
3                                                   5                                               
4                                                   3                                               
5                                                   5                                               
6                                                   5                                               
7                                                   3                                               
8                                                   4                                      

Definição de funções que ajudarão a ajustar o DataFrame:

In [118]:
def getIdFromColumnName(valor:str, nomeDaColuna: str, df: pd.DataFrame)->int :
    row = df.loc[df[nomeDaColuna]== valor]
    return row.iloc[0]['id']

print(getIdFromColumnName('Eu achei que responder à pesquisa de feedback de sessão de julgamento foi muito interessante.', 'perguntaAdaptada', dfPerguntasFiltradas))


5


In [119]:
def getTpById(valor:str,  df: pd.DataFrame)->int :
    return df.loc[df['id']== int(valor)].iloc[0]['tp']

print(getTpById('3',  dfPerguntasFiltradas))

R


Vamos modificar o nome das colunas para que correspondam ao ID do questionário IMI:

In [120]:
dfRespostasComId = dfRespostas.copy()
columnsDictionary = {}
for coluna in dfRespostas.columns:
    idColuna = str(getIdFromColumnName(coluna, 'perguntaAdaptada', dfPerguntasFiltradas))
    columnsDictionary[coluna] = idColuna

print(columnsDictionary)
dfRespostasComId.rename(columns = columnsDictionary, inplace=True)
print(dfRespostasComId)

{'Eu achei que responder à pesquisa de feedback de sessão de julgamento foi muito interessante.': '5', 'Acho que responder a pesquisa de feedback é útil para melhorar a atividade jurisdicional do TST.': '32', 'Senti que posso confiar nas pesquisa de feedback sobre sessões de julgamento do TST.': '40', 'Eu me senti pressionado(a) enquanto respondia a pesquisa de feedback para o TST.': '23', 'Foi prazeroso ajudar o TST com a pesquisa de feedback da sessão.': '1', 'Fiz um esforço considerável em responder a pesquisa de feedback para avaliar as sessões de julgamento do TST.': '14', 'Creio que responder a pesquisa de feedback para o TST pode me ajudar a ter uma melhor participação em sessões de julgamento.': '35', 'Respondi a pesquisa de feedback para o TST porque eu tinha que fazer isso.': '30', 'Achei monótono responder a pesquisa de feedback para o TST.': '3', 'Gostaria de interagir mais frequentemente com o TST.': '41', 'Sugerir melhorias com pesquisas de feedback para o TST não é algo 

Agora faremos os ajustes para as respostas que tem o Tp = 'R', o que indica que o valor deverá ser modificado para refletir a resposta ajustada. O primeiro passo será criar uma coluna com o valor da escala de Linkert utilizada (no nosso caso, 5) acrescido de 1, para que possamos subtrair essa coluna do nosso DataFrame:

In [121]:
# Criando uma coluna com valores 6 para permitir a subtração da coluna inteira
numeroRegistros =  dfRespostas.shape[0]
s = pd.Series(6)
listaComNumerosAjuste = s.repeat(numeroRegistros)
listaComNumerosAjuste = listaComNumerosAjuste.reset_index(drop=True)
print(listaComNumerosAjuste)

0     6
1     6
2     6
3     6
4     6
5     6
6     6
7     6
8     6
9     6
10    6
11    6
12    6
13    6
14    6
dtype: int64


Após isso, inspecionaremos as colunas para identificar aquelas que devem ser revertidas, ou seja, que contenham o Tp = R na planilha de perguntas do IMI

In [122]:

dfRespostasAjustadas = dfRespostasComId.copy()
for col in dfRespostasAjustadas.columns:
    if (col.isdigit() and getTpById(col, dfPerguntasFiltradas) == 'R'):
        print('Coluna '+ col+ ' tem o Tp = R')
        dfRespostasAjustadas[col] = listaComNumerosAjuste - dfRespostasAjustadas[col]
print('Respostas originais')
print(dfRespostasComId)
print('Respostas ajustadas')
print(dfRespostasAjustadas)

Coluna 30 tem o Tp = R
Coluna 3 tem o Tp = R
Coluna 13 tem o Tp = R
Coluna 43 tem o Tp = R
Respostas originais
    5  32  40  23  1  14  35  30  3  41  13  8  43  21  29
0   4   4   4   2  3   4   4   4  3   2   3  4   4   4   4
1   5   5   5   1  5   3   5   1  1   5   1  5   1   5   5
2   3   4   4   1  4   3   4   1  1   3   3  1   3   3   3
3   5   5   5   1  3   1   1   1  1   1   3  3   1   5   5
4   3   5   3   5  5   2   3   2  1   5   1  4   3   5   5
5   5   5   5   1  5   2   4   4  2   5   3  3   2   5   5
6   5   5   5   1  5   1   5   1  1   5   1  3   1   5   5
7   3   1   1   4  3   2   1   4  3   4   1  4   5   2   4
8   4   5   4   1  4   1   3   1  3   4   1  5   4   4   5
9   5   5   5   1  5   4   2   1  1   5   1  5   1   5   5
10  5   4   5   1  5   1   5   1  1   4   3  3   1   4   5
11  5   5   5   1  5   1   5   1  1   5   5  5   1   5   5
12  5   5   5   1  5   2   5   1  1   5   1  5   1   5   5
13  4   4   4   1  4   1   4   1  1   4   1  4   1   5   5
14  

Com esses valores, podemos fazer agora o cálculo do Alpha de Cronbach:

In [123]:
pg.cronbach_alpha(data=dfRespostasAjustadas)

(0.7770029884730325, array([0.569, 0.912]))